In [1]:
import sys
if '../' not in sys.path:
    sys.path.append('../')
import torch
import torch.nn as nn
from transformers import BertForMaskedLM,BertTokenizer
import random
from albumentations.core.transforms_interface import DualTransform, BasicTransform
from libnlp import transcription,env,preprocessing
from nltk.tokenize.treebank import TreebankWordDetokenizer 

In [2]:
import torch.nn.functional as F
import sys
import os
import numpy as np
from libnlp import preprocessing
import pandas as pd
from torch.utils.data import DataLoader,Dataset
from ignite.utils import convert_tensor
from pytorch_pretrained_bert.modeling import BertModel
import numpy as np


In [3]:
T=transcription.Transcribe()
E=env.Env()

Using cache found in C:\Users\Jamiu Afolabi/.cache\torch\hub\huggingface_pytorch-transformers_master
Using cache found in C:\Users\Jamiu Afolabi/.cache\torch\hub\huggingface_pytorch-transformers_master


In [4]:
ret,_=T.transcription_rule('rached hanbli klem maakoul win elli yefhem')

In [5]:
_

'راشد حانبلي كلم مااكول وين عللي يفحم'

# Exploring Dataset

In [7]:
train_df=pd.read_csv(os.path.join(E.ppd_path,E.train_path),encoding='utf-8')
train_df_idx=[i for i,text in enumerate(train_df.text.values) if len(text)>7]
#train_df=train_df.iloc[train_df_idx]
test_df=pd.read_csv(os.path.join(E.ppd_path,E.test_path),encoding='utf-8')

In [8]:
train_df['length_text']=train_df.text.apply(lambda x: len(x))

In [10]:
train_df[train_df.length_text < 7]

,text,label,length_text
22,b1 dit,1,6
784,vues,1,5
865,osb ca,2,6
947,1er vu,1,6
1191,tztm i,2,6
...,...,...,...
67933,ch 9al,0,6
68191,bara n,2,6
69384,et m6,1,5
69615,j amme,1,6


In [7]:
minority=train_df[train_df.label==0]
def ret_shortened_sentence(text):
    if len(text)>512:
        return text[:512]
    return text
minority['text']=minority.text.apply(lambda x: ret_shortened_sentence(x))

<ipython-input-7-0a3351425887>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minority['text']=minority.text.apply(lambda x: ret_shortened_sentence(x))


In [8]:
minority.to_csv('../Data/minority.csv',index=False)

In [10]:
sent_mask_1=[augmenter.apply(text) for text in minority.text]
sent_mask_2=[augmenter.apply(text,2) for text in minority.text]
sent_mask_3=[augmenter.apply(text,3,action='replace') for text in minority.text]
sent_mask_4=[augmenter.apply(text,4,action='insert') for text in minority.text]
sent_mask=np.hstack([sent_mask_1,sent_mask_2,sent_mask_3,sent_mask_4])
sent_mask=sent_mask.transpose()
new_train=pd.Dataframe()
new_train['text']=sent_mask
new_train['label']=0

IndexError: Cannot choose from an empty sequence

In [ ]:
new_train.to_csv('../Data/augmentation_minority.csv',index=False)